In [1]:
import pandas as pd

In [107]:
df = pd.read_csv('sp500-expanded.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.dropna(inplace=True)
one_hot = pd.get_dummies(df['activity_domain'], prefix='activity_domain')
df = df.drop('activity_domain', axis=1)
# Drop rows that have -1 values
df = df[~(df == -1).any(axis=1)]
df = pd.concat([df, one_hot], axis=1)
# high_cap_df = df[df['market_cap'] > 10000000000].copy()
# low_cap_df = df[df['market_cap'] <= 10000000000].copy()
high_cap_df, low_cap_df = [x for _, x in df.groupby(df['market_cap'] <= 10000000000)]
print(len(df1), len(df2))
columns_to_convert = [col for col in df.columns if 'activity_domain' in col]
columns_to_avoid = columns_to_convert + ['ticker']
normalization_columns = ['market_cap',
    'net_revenue_y1', 'net_revenue_y2',
    'net_revenue_y3',
    'net_revenue_y4', 
    'net_income_y1', 
    'net_income_y2',
    'net_income_y3',
    'net_income_y4',
    'free_cash_flow',
    'stock_price',]
for column in columns_to_convert:
    high_cap_df[column] = high_cap_df[column].astype(int)
    low_cap_df[column] = low_cap_df[column].astype(int)
for column in df.columns:
    if column not in columns_to_avoid:
        # if column in normalization_columns:
        #     df[column] = (df[column] - df[column].mean()) / df[column].std()
        # else:
        high_cap_df[column] = (high_cap_df[column] - high_cap_df[column].min()) / (high_cap_df[column].max() - high_cap_df[column].min())
        low_cap_df[column] = (low_cap_df[column] - low_cap_df[column].min()) / (low_cap_df[column].max() - low_cap_df[column].min())
train_df = high_cap_df.drop(columns=['ticker'])

4445 301


In [97]:
len(low_cap_df)

301

R LOOK AT DATA DISTRIBUTION

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_column_distributions(df):
    """
    Plot the distribution of each column in a dataframe.

    :param df: Input pandas DataFrame
    """
    
    # Number of columns in the dataframe
    num_cols = df.shape[1]

    # Setting up the subplots
    fig, axes = plt.subplots(nrows=num_cols, figsize=(8, 4*num_cols))

    # Iterating through each column to plot
    for ax, (col_name, col_data) in zip(axes, df.items()):
        if df[col_name].dtype.kind in 'bifc':  # Numerical types
            col_data.hist(ax=ax, bins=50, edgecolor='black')
            ax.set_title(f"Distribution of {col_name}")
            ax.set_xlabel(col_name)
            ax.set_ylabel('Frequency')
        else:
            # If non-numeric, we consider it categorical and use a bar plot
            col_data.value_counts().plot(kind='bar', ax=ax)
            ax.set_title(f"Distribution of {col_name}")
            ax.set_xlabel(col_name)
            ax.set_ylabel('Count')
        ax.set_xlim(left=-1, right=1)
    plt.tight_layout()
    plt.show()

plot_column_distributions(df)


# NETWORK

In [13]:
import sklearn
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import importlib


# Prepare data for training/testing

In [42]:
from sklearn.model_selection import train_test_split
import ml_backend.autoencoders
importlib.reload(ml_backend.autoencoders)
from ml_backend.autoencoders import Autoencoder

# Split the data into training and test sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Training

In [26]:
import torch.optim as optim
import torch
import torch.utils.data as data_utils
from hyperopt import hp, fmin, tpe, space_eval, Trials

def hyper_tune(params):
    model = Autoencoder(len(df.columns), params['first_layer'], params['latent_dim'], params['reduction'])
    match params['optim']:
        case "Adam":
            optim = torch.optim.Adam(model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])
        case "SGD":
            optim = torch.optim.SGD(model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])
        case _:
            raise ValueError("Invalid optimizer")
    loss_f = params['loss']
    data_loader = DataLoader(train_data, batch_size=32, shuffle=False)
    for epoch in range(params['epochs']):
        epoch_loss = 0
        for batch_data, _ in data_loader:
            optim.zero_grad()
            reconstructed = model(batch_data)
            loss = loss_f(reconstructed, batch_data)
            loss.backward()
            optim.step()
            epoch_loss += loss.item() * len(batch_data)
        epoch_loss /= len(train_data.tensors[0])
        print(f"Epoch {epoch} loss: {epoch_loss}")
    print("Training finished")
    return epoch_loss


lr = hp.loguniform("lr", -5, 0)
weight_decay = hp.loguniform("weight_decay", -8, -4)
latent_dim = hp.choice("latent_dim", [2, 3, 4, 5, 6, 7, 8, 9, 10])
space = {
    "reduction": hp.uniform("reduction", 1.3, 10),
    "lr": lr,
    "optim": hp.choice("optim", ["Adam", "SGD"]),
    "first_layer": hp.choice("first_layer", range(25, 500)),
    "latent_dim": hp.choice("latent_dim", range(2, 10)),
    "loss": hp.choice("loss", [nn.MSELoss(), nn.L1Loss()]),
    "weight_decay": weight_decay,
    "epochs": hp.choice("epochs", range(1, 16)),
}

train_data = torch.tensor(train_df.values, dtype=torch.float32)
train_data = data_utils.TensorDataset(train_data, traax_evals=100, trials=Trials())

i_size: 25, o_size: 17                                 
i_size: 17, o_size: 11                                 
i_size: 11, o_size: 8                                  
i_size: 8, o_size: 5                                   
i_size: 5, o_size: 3                                   
i_size: 3, o_size: 2                                   
[torch.Size([25, 51]), torch.Size([17, 25]), torch.Size([11, 17]), torch.Size([8, 11]), torch.Size([5, 8]), torch.Size([3, 5]), torch.Size([2, 3])]
[torch.Size([3, 2]), torch.Size([5, 3]), torch.Size([8, 5]), torch.Size([11, 8]), torch.Size([17, 11]), torch.Size([25, 17]), torch.Size([51, 25])]
Epoch 0 loss: 0.6799255509602942                       
Epoch 1 loss: 0.1177224273062407                       
Training finished                                      
i_size: 407, o_size: 112                                                         
i_size: 112, o_size: 30                                                          
i_size: 30, o_size: 8               

In [39]:
print(best)

{'epochs': 9, 'first_layer': 415, 'latent_dim': 0, 'loss': 0, 'lr': 0.023592868709587128, 'optim': 0, 'reduction': 5.201152361216245, 'weight_decay': 0.004576021780115587}


In [44]:
import numpy as np 
lr = best['lr']
weight_decay = best['weight_decay']
latent_dim = np.arange(2, 10, 1)[best['latent_dim']]
first_layer_size = np.arange(25, 500, 1)[best['first_layer']]
reduction = best['reduction']
loss_f = torch.nn.MSELoss() if best['loss'] == 0 else torch.nn.L1Loss()  
epochs = np.arange(1, 16, 1)[best['epochs']]
model = Autoencoder(len(df.columns), first_layer_size, latent_dim, reduction)
print(best['optim'])
optim = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay) if best['optim'] == 0 else torch.optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
model.train(train_data, loss_f, optim, n_epochs=epochs)
torch.save(model.state_dict(), 'model.pt')

i_size: 440, o_size: 84
i_size: 84, o_size: 16
i_size: 16, o_size: 3
i_size: 3, o_size: 0
[torch.Size([440, 51]), torch.Size([84, 440]), torch.Size([16, 84]), torch.Size([3, 16]), torch.Size([2, 3])]
[torch.Size([3, 2]), torch.Size([16, 3]), torch.Size([84, 16]), torch.Size([440, 84]), torch.Size([51, 440])]
0
Epoch 0 loss: 0.030550536855168692
Epoch 1 loss: 0.019408496599687047
Epoch 2 loss: 0.01947394385081407
Epoch 3 loss: 0.019269858608427254
Epoch 4 loss: 0.019252870956465638
Epoch 5 loss: 0.018879635915910353
Epoch 6 loss: 0.019074281882196325
Epoch 7 loss: 0.018915201309275573
Epoch 8 loss: 0.019019032191289097
Epoch 9 loss: 0.018702203633787974
Training finished


# Testing

In [45]:
low_market_cap_df = df[df['market_cap'] < pow(10, 9)]
low_market_cap_data = torch.tensor(low_market_cap_df.values, dtype=torch.float32)
val_data = torch.tensor(val_df.values, dtype=torch.float6432)

## See loss for validation set of high-cap stocks

In [77]:
losses = []
loss = 0
for data_point in val_data:
    loss += loss_f(model(data_point), data_point)
    losses.append(loss.item())
loss /= len(low_market_cap_data)
print("Average loss for validation set: ", loss)
    

Average loss for validation set:  tensor(0.0037, grad_fn=<DivBackward0>)


## See loss for low-cap stocks

In [76]:
low_losses = []
low_loss = 0
for data_point in low_market_cap_data:
    low_loss += loss_f(model(data_point), data_point)
    low_losses.append(low_loss.item())
low_loss /= len(low_market_cap_data)
print("Average loss for low market cap set: ", low_loss)

Average loss for low market cap set:  tensor(0.0186, grad_fn=<DivBackward0>)


### If loss of stock reconstruction is smaller than average loss of validation(maybe all stocks with big market cap here but not sure since some of them are used for training) then stock is good, otherwise stock bad

,ticker,market_cap,net_revenue_y1,net_revenue_y2,net_revenue_y3,net_revenue_y4,net_income_y1,net_income_y2,net_income_y3,net_income_y4,...,activity_domain_Communication Services,activity_domain_Consumer Cyclical,activity_domain_Consumer Defensive,activity_domain_Energy,activity_domain_Financial Services,activity_domain_Healthcare,activity_domain_Industrials,activity_domain_Real Estate,activity_domain_Technology,activity_domain_Utilities
0,A,0.013270,0.002755,0.003964,0.004264,0.004837,0.003176,0.012401,0.119768,0.092641,...,0,0,0,0,0,1,0,0,0,0
14,AAPL,1.000000,0.158733,0.230435,0.205200,0.243165,0.099934,0.169470,0.216260,0.235483,...,0,0,0,0,0,0,0,0,1,0
18,ABBV,0.096133,0.023368,0.035386,0.034470,0.031102,0.013566,0.029763,0.126401,0.110596,...,0,0,0,0,0,1,0,0,0,0
24,ABEV,0.015999,0.032085,0.045879,0.043857,0.049170,0.016140,0.031660,0.137913,0.120872,...,0,0,1,0,0,0,0,0,0,0
28,ABM,0.001051,0.003141,0.003907,0.004748,0.006085,0.002171,0.010580,0.118545,0.090154,...,0,0,0,0,0,0,1,0,0,0
